In [1]:
# Daily Invest AI Demo

In [137]:
# libraries
from nsepython import *
from datetime import datetime, timedelta
import pandas as pd

import ollama
import chromadb

from tqdm import tqdm

### Step 1: Loading the latest news

In [138]:
## Filtering only nse listed stocks i.e 760
file_path = "../data/all_companies_nse.xlsx"
nse_listed = pd.read_excel(file_path)

C:\Users\yeshu\.conda\envs\ai\lib\site-packages\openpyxl\descriptors\base.py:47: RuntimeWarning: coroutine 'translate_text' was never awaited
  super(Typed, self).__set__(instance, value)


In [139]:
nse_listed.columns = [c.lower() for c in nse_listed.columns]

In [140]:
news_file = "latest_news.csv"
news = pd.read_csv(news_file,encoding='cp1252')

In [144]:
news.shape

(49, 4)

In [143]:
news.dropna(inplace=True)

In [145]:
name_symbol_map = {}
for arr in nse_listed[['company name', 'symbol']].values:
    name_symbol_map[arr[0]]=arr[1]

In [146]:
## adding few others :>
name_symbol_map['Caplin Point Laboratories'] = "CAPLIPOINT"
name_symbol_map['PERSISTENT SYSTEMS'] = "PERSISTENT"
name_symbol_map['MACROTECH DEVELOPERS'] = "LODHA"
name_symbol_map['KNR CONSTRUCTIONS'] = "KNRCON"
name_symbol_map['KALYAN JEWELLERS'] = "KALYANKJIL"

In [16]:
updated_records = []

In [18]:
## Adding company Name :>
news_records = news.to_dict("records")

In [ ]:
chroma_client = chromadb.PersistentClient(path="names_db")

In [21]:
# Creating 
all_company_names = list(nse_listed['company name'].values)
collection_names = chroma_client.create_collection(name="nse_listed_company_names")

collection_names.add(
    documents=all_company_names,
    ids=[str(i) for i in range(len(all_company_names))]
)


In [22]:
collection_names = chroma_client.get_collection(name="nse_listed_company_names")

In [23]:
## Get Name function
# Chroma 
# llm llava

def processName(res):
    try:
        
        res = res.replace("json","")
        res = res.replace("`","")
        res = res.strip()
        res = eval(res)
        name = res['company']
    except:
        print("Issue with :>",res)
        name = ""
    return name
    
def extractCompanyName(n):

    results = collection_names.query(
        query_texts=[n], # Chroma will embed this for you
        n_results=5 # how many results to return
    )

    choices = "\n" + "\n".join(results['documents'][0]) + "\n"

    ## Asking LLM for further identification :>
    sample_response = {"company":""}
    prompt = f"""You are an expert financial analyst. You are expert in identification of company in the news article. 
    You are given an news article. You need to identify which company from the choices below mentioned in the news article.
    Note: Company names might be in short form, as it is a twitter text post. Match the name even for fuzzy matching. 
    
    
    Here is the news article : {n}
    
    Here are the choices : {choices}
    
    provide the answer in json format {sample_response}
    Dont change the name of the company from the choices even the . or , or casing.
    Cross check once again, dont mention reasoning or change the names of the choices. Just give me the company name in above format
    """

    response = ollama.chat(
        model='llava:7b',
        messages=[{
            'role': 'user',
            'content': prompt
        }]
    )
    
    res = response['message']['content']

    company_name = processName(res)
    
    return company_name
    

In [40]:
for r in tqdm(news_records[4+19+1+5+1+6+5+1:]):
    obj = r.copy()
    company_name = extractCompanyName(obj['text'])
    
    company_symbol = name_symbol_map[company_name]
    obj["company name"] = company_name
    obj["symbol"] = company_symbol
    updated_records.append(obj)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:26<00:00,  4.45s/it]


In [135]:
updated_records[-1]

{'user': 'REDBOXINDIA',
 'id': 1876267042678190355,
 'Date&Time': '2025-01-06T13:58:16.000Z',
 'text': 'NESCO: CO DECLARED HIGHEST BIDDER FOR WAYSIDE AMENITIES ON BENGALURU CHENNAI EXPRESSWAY || TOTAL DEVELOPMENT COST ESTIMATED AT 750M RUPEES PER SITE; ESTIMATED ANNUALISED REVENUE 3.50B RUPEES FROM YEAR 4',
 'company name': 'NESCO Ltd.',
 'symbol': 'NESCO'}

## Translation

In [66]:
sample_format = {
    "Hindi": "",
    "German": ""
}

In [67]:
news_article = updated_records[-1]['text']

In [68]:
## Now Transalating to each language :> 
prompt = f"""You are expert in transalting a sentence from english to other languages. You were given an news articles in short \
form in english. Translate it to following languages:
Hindi, German
Here is the new article: {news_article}
note: Donot change the meaning of the news article. Do not change the company names in the news. 
Provide the output in following format. Only provide in following json format. Donot include even phrases like "here it is"
:> {sample_format}
"""

In [69]:
response = ollama.chat(
        model='llama3.2:latest',
        messages=[{
            'role': 'user',
            'content': prompt
        }]
    )
    
res = response['message']['content']


In [70]:
res

'>\n{\n    "Hindi": "नेस्को : कॉ Declared Highest Bidder For Wayside Amenities On Bengaluru Chennai Expressway || Total Development Cost Estimated At 750 M Rupees Per Site; Estimated Annualised Revenue 3.50 B Rupees From Year 4",\n    "German": "NESCO: CO ist höchstgebidzener für Wegebegünstigungen auf der Bengalur-Chennai-Expressway || Gesamtkostenabschätzung für einen Standort bei 750 Mio. Euro; Schätzungsrevenue pro Jahr von 3,5 Milliarden Rupees ab Jahresbeginn"\n}'

In [71]:
# Formating :
def processText(res):
    index_left = res.index("{")
    index_right = res.index("}")

    res_text = res[index_left:index_right+1]
    res_obj = eval(res_text)
    return res_obj

In [72]:
processText(res)

{'Hindi': 'नेस्को : कॉ Declared Highest Bidder For Wayside Amenities On Bengaluru Chennai Expressway || Total Development Cost Estimated At 750 M Rupees Per Site; Estimated Annualised Revenue 3.50 B Rupees From Year 4',
 'German': 'NESCO: CO ist höchstgebidzener für Wegebegünstigungen auf der Bengalur-Chennai-Expressway || Gesamtkostenabschätzung für einen Standort bei 750 Mio. Euro; Schätzungsrevenue pro Jahr von 3,5 Milliarden Rupees ab Jahresbeginn'}

In [75]:
final_records = []

In [97]:
for r in tqdm(updated_records[1+3+12+1+1+1+17+3:]):
    obj = r.copy()
    news_article = obj['text'].replace("\n", " ")
    ## Now Transalating to each language :> 
    prompt = f"""You are expert in transalting a sentence from english to other languages. You were given an news articles in short \
    form in english. Translate it to following languages:
    Hindi, German
    Here is the new article: {news_article}
    note: Donot change the meaning of the news article. Do not change the company names in the news. 
    Provide the output in following format. Only provide in following json format. Donot include even phrases like "here it is"
    :> {sample_format}
    """

    response = ollama.chat(
            model='llama3.2:latest',
            messages=[{
                'role': 'user',
                'content': prompt
            }]
        )
        
    res = response['message']['content']
    res_trans = processText(res)
    obj.update(res_trans)
    final_records.append(obj)
        

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:40<00:00, 14.43s/it]


In [98]:
final_records[-1]

{'user': 'REDBOXINDIA',
 'id': 1876267042678190355,
 'Date&Time': '2025-01-06T13:58:16.000Z',
 'text': 'NESCO: CO DECLARED HIGHEST BIDDER FOR WAYSIDE AMENITIES ON BENGALURU CHENNAI EXPRESSWAY || TOTAL DEVELOPMENT COST ESTIMATED AT 750M RUPEES PER SITE; ESTIMATED ANNUALISED REVENUE 3.50B RUPEES FROM YEAR 4',
 'company name': 'NESCO Ltd.',
 'symbol': 'NESCO',
 'Hindi': 'नाभी गॉलैः ऴॖইৱ अा ॸজদ फाी अभ्पेली',
 'German': 'Фор Персеко: ко лубощня компличер ваяс фоур мёестеи кны ваще турю Сялдянмют фі ка шгое ардъ, у \x04честняй \\u0430нувялитиз \x05ръвеню \x06ьтё мфлмяРягю 3.50 Искрх \\u044eля \x07ряды \\8юркща'}

In [99]:
news_article

'NESCO: CO DECLARED HIGHEST BIDDER FOR WAYSIDE AMENITIES ON BENGALURU CHENNAI EXPRESSWAY || TOTAL DEVELOPMENT COST ESTIMATED AT 750M RUPEES PER SITE; ESTIMATED ANNUALISED REVENUE 3.50B RUPEES FROM YEAR 4'

In [100]:
res_trans

{'Hindi': 'नाभी गॉलैः ऴॖইৱ अा ॸজদ फाी अभ्पेली',
 'German': 'Фор Персеко: ко лубощня компличер ваяс фоур мёестеи кны ваще турю Сялдянмют фі ка шгое ардъ, у \x04честняй \\u0430нувялитиз \x05ръвеню \x06ьтё мфлмяРягю 3.50 Искрх \\u044eля \x07ряды \\8юркща'}

In [105]:
!pip install googletrans

  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio-client 0.15.0 requires websockets<12.0,>=10.0, but you have websockets 12.0 which is incompatible.
llama-index-readers-file 0.1.19 requires pypdf<5.0.0,>=4.0.1, but you have pypdf 5.1.0 which is incompatible.
ollama 0.3.3 requires httpx<0.28.0,>=0.27.0, but you have httpx 0.28.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [106]:
import asyncio
from googletrans import Translator

In [115]:
translator = Translator(service_urls=[
      'translate.google.com',
    ])

In [116]:
result = translator.translate([news_article], dest='hi')

In [118]:
from googletrans import Translator
translator = Translator()
translator.translate('안녕하세요.')

<coroutine object Translator.translate at 0x0000023FBA29F220>

In [119]:
async def translate_text():
    async with Translator() as translator:
        result = await translator.translate('안녕하세요.')
        print(result)  # <Translated src=ko dest=en text=Good evening. pronunciation=Good evening.>

asyncio.run(translate_text())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [155]:
import nest_asyncio
nest_asyncio.apply()

In [153]:
import asyncio

async def main():
    result = await translator.translate(news_article, dest='de')
    print(result.text)
    
asyncio.run(main())

NESCO: CO ERKLÄRT HÖCHSTEN BIETER FÜR ANNEHMLICHKEITEN AM BENGALURU CHENNAI EXPRESSWAY || Die gesamten Entwicklungskosten werden auf 750 Millionen Rupien pro Standort geschätzt; GESCHÄTZTER JAHRESUMSATZ 3,50 MILLIARDEN RUPIEN AB JAHR 4


In [133]:
news_article

'NESCO: CO DECLARED HIGHEST BIDDER FOR WAYSIDE AMENITIES ON BENGALURU CHENNAI EXPRESSWAY || TOTAL DEVELOPMENT COST ESTIMATED AT 750M RUPEES PER SITE; ESTIMATED ANNUALISED REVENUE 3.50B RUPEES FROM YEAR 4'

In [154]:
updated_records[0]

{'user': 'REDBOXINDIA',
 'id': 1876225896090358016,
 'Date&Time': '2025-01-06T11:14:46.000Z',
 'text': 'S H KELKAR AND CO: CO SAYS CONSOLIDATED REVENUES FOR 9M FY2025 AT 15.48B RUPEES, UP 17% (YOY) || GROSS MARGINS UNDER PRESSURE PRIMARILY DUE TO SUPPLY CONSTRAINTS; ON TRACK TO ACHIEVE DOUBLE-DIGIT GROWTH FOR FULL FINANCIAL YEAR',
 'company name': 'Banco Products (India) Ltd.',
 'symbol': 'BANCOINDIA'}

In [136]:
len(updated_records)

46

In [148]:
### Translation into Hindi and Germany

In [166]:
import asyncio
final_records = []

In [171]:


async def main():

    for r in tqdm(updated_records):
        obj = r.copy()
        n = obj['text']
        
        hindi = await translator.translate(n, dest='hi')
        telugu = await translator.translate(n, dest='te')
        
        obj["hindi"] = hindi.text
        obj["telugu"] = telugu.text
        
        final_records.append(obj)
        
    print("done!!")
    

In [172]:
asyncio.run(main())

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:19<00:00,  2.34it/s]

done!!


In [173]:
final_records

[{'user': 'REDBOXINDIA',
  'id': 1876225896090358016,
  'Date&Time': '2025-01-06T11:14:46.000Z',
  'text': 'S H KELKAR AND CO: CO SAYS CONSOLIDATED REVENUES FOR 9M FY2025 AT 15.48B RUPEES, UP 17% (YOY) || GROSS MARGINS UNDER PRESSURE PRIMARILY DUE TO SUPPLY CONSTRAINTS; ON TRACK TO ACHIEVE DOUBLE-DIGIT GROWTH FOR FULL FINANCIAL YEAR',
  'company name': 'Banco Products (India) Ltd.',
  'symbol': 'BANCOINDIA',
  'hindi': 'एस एच केलकर एंड कंपनी: कंपनी का कहना है कि वित्तीय वर्ष 2025 के 9 महीने के लिए समेकित राजस्व 15.48 अरब रुपये, 17% (वर्ष-दर-वर्ष) ऊपर है || सकल मार्जिन मुख्यतः आपूर्ति बाधाओं के कारण दबाव में है; पूरे वित्तीय वर्ष में दोहरे अंक की वृद्धि हासिल करने की राह पर',
  'telugu': 'S H కేల్కర్ మరియు CO: CO 9M FY2025 కోసం 15.48B రూపాయలకు ఏకీకృత ఆదాయాలను 17% (YOY) తెలిపింది || ఒత్తిడిలో స్థూల మార్జిన్లు ప్రధానంగా సరఫరా పరిమితుల కారణంగా; పూర్తి ఆర్థిక సంవత్సరానికి రెండంకెల వృద్ధిని సాధించడానికి ట్రాక్\u200cలో ఉంది'},
 {'user': 'REDBOXINDIA',
  'id': 1876193146155172341,
  'Date&Time

In [169]:
len(updated_records)

46

In [170]:
updated_records[0]

{'user': 'REDBOXINDIA',
 'id': 1876225896090358016,
 'Date&Time': '2025-01-06T11:14:46.000Z',
 'text': 'S H KELKAR AND CO: CO SAYS CONSOLIDATED REVENUES FOR 9M FY2025 AT 15.48B RUPEES, UP 17% (YOY) || GROSS MARGINS UNDER PRESSURE PRIMARILY DUE TO SUPPLY CONSTRAINTS; ON TRACK TO ACHIEVE DOUBLE-DIGIT GROWTH FOR FULL FINANCIAL YEAR',
 'company name': 'Banco Products (India) Ltd.',
 'symbol': 'BANCOINDIA'}

In [179]:
## Adding latest price
final_records_price = []
for r in tqdm(final_records):
    obj = r.copy()
    sym = obj['symbol']
    obj['price'] =  nse_eq(sym)['priceInfo']['lastPrice']
    final_records_price.append(obj)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [02:21<00:00,  3.08s/it]


In [180]:
final_records_price[0]

{'user': 'REDBOXINDIA',
 'id': 1876225896090358016,
 'Date&Time': '2025-01-06T11:14:46.000Z',
 'text': 'S H KELKAR AND CO: CO SAYS CONSOLIDATED REVENUES FOR 9M FY2025 AT 15.48B RUPEES, UP 17% (YOY) || GROSS MARGINS UNDER PRESSURE PRIMARILY DUE TO SUPPLY CONSTRAINTS; ON TRACK TO ACHIEVE DOUBLE-DIGIT GROWTH FOR FULL FINANCIAL YEAR',
 'company name': 'Banco Products (India) Ltd.',
 'symbol': 'BANCOINDIA',
 'hindi': 'एस एच केलकर एंड कंपनी: कंपनी का कहना है कि वित्तीय वर्ष 2025 के 9 महीने के लिए समेकित राजस्व 15.48 अरब रुपये, 17% (वर्ष-दर-वर्ष) ऊपर है || सकल मार्जिन मुख्यतः आपूर्ति बाधाओं के कारण दबाव में है; पूरे वित्तीय वर्ष में दोहरे अंक की वृद्धि हासिल करने की राह पर',
 'telugu': 'S H కేల్కర్ మరియు CO: CO 9M FY2025 కోసం 15.48B రూపాయలకు ఏకీకృత ఆదాయాలను 17% (YOY) తెలిపింది || ఒత్తిడిలో స్థూల మార్జిన్లు ప్రధానంగా సరఫరా పరిమితుల కారణంగా; పూర్తి ఆర్థిక సంవత్సరానికి రెండంకెల వృద్ధిని సాధించడానికి ట్రాక్\u200cలో ఉంది',
 'price': 484}

In [184]:
import json
with open("final.json","w",encoding='utf-8') as f:
    json.dump(final_records_price,f,ensure_ascii=False)